In [1]:
import geopandas
import pandas as pd
import numpy as np
import random
import math
import seaborn as sns
import matplotlib.pyplot as plt

# Data

In [2]:
buildings = geopandas.read_file('data/by_get.shp')
buildings = buildings.to_crs("EPSG:4326")
buildings.geometry = buildings.geometry.centroid

/tmp/ipykernel_7614/2813531054.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings.geometry = buildings.geometry.centroid


In [3]:
ruta1000 = geopandas.read_file('data/B1 ruta 1000 difference with 250.shp')
ruta1000 = ruta1000.to_crs("EPSG:4326")

ruta250 = geopandas.read_file('data/B1 ruta 250.shp')
ruta250 = ruta250.to_crs("EPSG:4326")

rutas = geopandas.GeoDataFrame(pd.concat([ruta250, ruta1000], axis=0))

rutas['Ruta'] = rutas.Ruta.astype(int)
rutas = rutas.set_index('Ruta')

In [4]:
houses = buildings[buildings.ANDAMAL_1 < 200]
workplaces = geopandas.read_file('data/workplaces_OSM_SCB_merged.geojson')

In [ ]:
houses_in_ruta = []
workplaces_in_ruta = []
columns = rutas.reset_index().columns
r = rutas
for ruta in r.itertuples():
    ruta_repeated = geopandas.GeoDataFrame(np.repeat([ruta], len(houses.index), axis=0), columns=columns, geometry='geometry', crs="EPSG:4326")
    ruta_houses_ind = houses[ruta_repeated.contains(houses.geometry, align=False).values].index
    houses_in_ruta.append(ruta_houses_ind.to_list())
    
    ruta_repeated = geopandas.GeoDataFrame(np.repeat([ruta], len(workplaces.index), axis=0), columns=columns, geometry='geometry', crs="EPSG:4326")
    ruta_workplaces_ind = workplaces[ruta_repeated.contains(workplaces.geometry, align=False).values].index
    workplaces_in_ruta.append(ruta_workplaces_ind.to_list())

In [74]:
r['houses_in_ruta'] = houses_in_ruta
r['workplaces_in_ruta'] = workplaces_in_ruta

In [75]:
r

,Rutstorl,Ald0_6,Ald7_15,Ald16_19,Ald20_24,Ald25_44,Ald45_64,Ald65_W,Totalt,geometry,houses_in_ruta,workplaces_in_ruta
Ruta,,,,,,,,,,,,
4800006223750,250,0,3,0,3,3,11,3,23,"POLYGON ((14.67800 56.15835, 14.67798 56.16059...","[1074531, 1074534, 1074538, 1074542, 1074543, ...",[]
4797506223750,250,0,0,0,3,4,14,7,28,"POLYGON ((14.67398 56.15834, 14.67396 56.16058...","[1074615, 1074616, 1074617, 1074666, 1074685, ...",[]


In [36]:
r.drop('geometry', axis=1).to_feather('data/rutas_with_buildings.feather')

/tmp/ipykernel_25569/4042759684.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  r.drop('geometry', axis=1).to_feather('data/rutas_with_buildings.feather')


In [38]:
pd.read_feather('data/rutas_with_buildings.feather')